# Задача потребителя

## Получение данных

Параметры модулирования:


>  Станция
    Novosibirsk(MRK)
    
    X
    452.28865
    Y
    3635.913158
    Z
    5203.397861

>  Спутники

*   1. 1/2
*   2. 2/1
*   3. 3/12
*   4. 4/6


Выполним импорты нужных модулей

In [0]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

Запомним координаты станции как вектор для проверки в конце

In [0]:
 required_coordinates = np.array([
        452.28865,
        3635.913158,
        5203.397861,
 ])

Загрузим excel-файл с данными модуляции

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('Загруженный "{name}" размером {length} байт'.format(
      name=fn, length=len(uploaded[fn])))

Saving reports.xls to reports (1).xls
Загруженный "reports.xls" размером 306688 байт


Откроем файл в виде таблицы, чекнем превью

In [6]:
mod_data = pd.read_excel("reports.xls")
mod_data.head()

,Time_mom,N_Sat,P1,Sat_X,Sat_Y,Sat_Z
0,01:47:00,1,23640.463423,-1403.143993,25077.821790,-4577.880104
1,01:47:30,1,23614.250832,-1400.494773,25096.862175,-4472.742878
2,01:48:00,1,23588.052424,-1397.739724,25115.469797,-4367.509327
3,01:48:30,1,23561.868740,-1394.880788,25133.643881,-4262.181717
4,01:49:00,1,23535.700307,-1391.919910,25151.383669,-4156.762322


найдем моменты, когда одновременно в зоне видимости хотя бы три спутника

In [7]:
by_time = mod_data.groupby(by='Time_mom')['N_Sat'].count() 
observed_more_than_two = by_time[by_time>2]
len(observed_more_than_two)

405

таких моментов около 400, а вот сразу четыре спутника никогда не видны

In [8]:
len(observed_more_than_two[observed_more_than_two>3])   # одновременно 4 никогда не наблюдаются

0

возьмем какой-нибудь рандомный момент

In [9]:
sample_time = observed_more_than_two.index[78]
sample = mod_data[mod_data['Time_mom'] == sample_time]
sample

,Time_mom,N_Sat,P1,Sat_X,Sat_Y,Sat_Z
785,05:25:00,2,19052.577323,2184.905275,15349.310978,20129.689073
786,05:25:00,1,21106.787484,-11915.881636,3162.669722,22300.104699
787,05:25:00,3,21880.141257,16211.479975,18786.485990,6124.669437


## Предустановки

Выделение из записи таблицы вектора координат спутника и расстояния

In [0]:
def get_Us_and_P(row):
    return np.array([row['Sat_X'],
                     row['Sat_Y'],
                     row['Sat_Z'],]), row['P1']

Евклидово расстояние между двумя векторами

In [0]:
def distance(v1,v2):
    return np.sqrt(np.sum((v1-v2)**2))

Так будем рассчитывать направляющие косинусы по три за раз (все парамеры - векторы)

In [0]:
def dir_cos(Us,Ur,Ro0):
    return (Us-Ur)/Ro0

## I способ расчета

Выделим нужные данные по спутникам из образца

In [13]:
sat_data = [get_Us_and_P(sample.iloc[i]) for i in range(3)]
sat_data

[(array([ 2184.90527483, 15349.3109776 , 20129.68907288]), 19052.5773226953),
 (array([-11915.8816357 ,   3162.66972196,  22300.10469944]), 21106.787484125),
 (array([16211.47997461, 18786.48599038,  6124.66943681]), 21880.1412569171)]

Так выглядит система уравнений, где неизвестные - координаты потребителя. Уравнения должны быть приведены к нулю.

In [0]:
def to_solve_Ur_0(v,Sat_data):
    return [distance(v,u) - p for u,p in Sat_data]

При подстановке в систему искомых координат результат должен быть нулевым для каждого уравнения.

In [15]:
to_solve_Ur_0(required_coordinates,sat_data)

[-0.02321069562822231, -0.0828553766150435, 0.002930710372311296]

Посмотрим сколько времени займет решение системы

In [16]:
%time Ur_0 = fsolve(to_solve_Ur_0, np.array([1.,1.,1.]), args=(sat_data))

Ur_0              

CPU times: user 1.37 ms, sys: 0 ns, total: 1.37 ms
Wall time: 3.64 ms


array([ 450.6806568 , 3637.66420946, 5202.18098915])

Разделим координаты и расстояния в  данных спутников. Введем опорные расстояния - с обрезанной дробной частью

In [17]:
Ro = np.array([P_i for Us_i,P_i in sat_data])
Ro_0 = Ro.astype(int).astype(np.float64)
Us = [Us_i for Us_i,P in sat_data]
Us

[array([ 2184.90527483, 15349.3109776 , 20129.68907288]),
 array([-11915.8816357 ,   3162.66972196,  22300.10469944]),
 array([16211.47997461, 18786.48599038,  6124.66943681])]

Ищем направляющие косинусы

In [18]:
dir_cosines = [dir_cos(*vals) for vals in zip(Us,[Ur_0]*3,Ro_0)]
dir_cosines

[array([0.09102586, 0.61472007, 0.78351397]),
 array([-0.58592639, -0.02250519,  0.81009778]),
 array([0.72032904, 0.69235931, 0.04216126])]

Так выглядит система уравнений для нахождения дельт.

In [0]:
def to_solve_deltas(deltas,cosines,ro_0,ro):
    return [(-p+p0+np.sum(np.array(deltas)*cos)) for cos,p0,p in zip(cosines,ro_0,ro)]

In [20]:
to_solve_deltas([.0,.0,.0,],dir_cosines,Ro_0,Ro)

[-0.5773226952987898, -0.787484125001356, -0.14125691710069077]

Ищем дельты

In [21]:
%time deltas = fsolve(to_solve_deltas,[.0,.0,.0],args=(dir_cosines,Ro_0,Ro))
deltas

CPU times: user 980 µs, sys: 0 ns, total: 980 µs
Wall time: 1.12 ms


array([ 8.81696817, -9.40075801,  7.08804881])

Смотрим результаты

In [22]:
print('Без смещений')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0)
print('разности в координатах\n', abs(required_coordinates - Ur_0))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0))

Без смещений
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 450.6806568  3637.66420946 5202.18098915]
разности в координатах
 [1.6079932  1.75105146 1.21687185]
расстояние между реальной и предполагаемой точками
 2.6706928744430445


In [23]:
print('Со смещениями + delta')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0+deltas)
print('разности в координатах\n', abs(required_coordinates - Ur_0-deltas))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0+deltas))

Со смещениями + delta
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 459.49762497 3628.26345144 5209.26903796]
разности в координатах
 [7.20897497 7.64970656 5.87117696]
расстояние между реальной и предполагаемой точками
 12.039852551265334


In [24]:
print('Со смещениями - delta')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0-deltas)
print('разности в координатах\n', abs(required_coordinates - Ur_0+deltas))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0-deltas))

Со смещениями - delta
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 441.86368863 3647.06496747 5195.09294034]
разности в координатах
 [10.42496137 11.15180947  8.30492066]
расстояние между реальной и предполагаемой точками
 17.37856096396465


## II способ расчета

Воспользуемся некоторыми переменными, полученными ранее

In [25]:
print(sat_data)
print(Us)
print(Ro)

[(array([ 2184.90527483, 15349.3109776 , 20129.68907288]), 19052.5773226953), (array([-11915.8816357 ,   3162.66972196,  22300.10469944]), 21106.787484125), (array([16211.47997461, 18786.48599038,  6124.66943681]), 21880.1412569171)]
[array([ 2184.90527483, 15349.3109776 , 20129.68907288]), array([-11915.8816357 ,   3162.66972196,  22300.10469944]), array([16211.47997461, 18786.48599038,  6124.66943681])]
[19052.5773227  21106.78748413 21880.14125692]


Решая систему уравнений, ищем координаты потребителя. Берем только целочисленную часть и называем их опорными координатами.

In [26]:
%time Ur_0 = fsolve(to_solve_Ur_0, np.array([1.,1.,1.]), args=(sat_data)).astype(int).astype(np.float64)
Ur_0              

CPU times: user 814 µs, sys: 984 µs, total: 1.8 ms
Wall time: 2 ms


array([ 450., 3637., 5202.])

Вводим опорные расстояния как евклидовы расстояния между неточными координатами потребителя и координатами каждого спутника

In [27]:
Ro_0 = np.array([distance(Ur_0,Us_i) for Us_i in Us])
Ro_0

array([19053.18938669, 21106.52037058, 21881.09905038])

Ищем направляющие косинусы

In [28]:
dir_cosines = [dir_cos(*vals) for vals in zip(Us,[Ur_0]*3,Ro_0)]
dir_cosines

[array([0.0910559 , 0.61471656, 0.78347455]),
 array([-0.58587969, -0.02247316,  0.81008638]),
 array([0.72032396, 0.69235489, 0.04216742])]

Посмотрим, сколько времени займет решение системы уравнений для поиска дельт

In [29]:
%time deltas = fsolve(to_solve_deltas,[.0,.0,.0],args=(dir_cosines,Ro_0,Ro))
deltas

CPU times: user 1.1 ms, sys: 0 ns, total: 1.1 ms
Wall time: 1.54 ms


array([-0.6805129 , -0.66436613, -0.1808641 ])

Смотрим результаты для того самого образца

In [30]:
print('Без смещений')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0)
print('разности в координатах\n', abs(required_coordinates - Ur_0))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0))

Без смещений
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 450. 3637. 5202.]
разности в координатах
 [2.28865  1.086842 1.397861]
расстояние между реальной и предполагаемой точками
 2.8936412581359288


In [31]:
print('Со смещениями - delta')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0-deltas)
print('разности в координатах\n', abs(required_coordinates - Ur_0+deltas))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0-deltas))

Со смещениями - delta
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 450.6805129  3637.66436613 5202.1808641 ]
разности в координатах
 [1.6081371  1.75120813 1.2169969 ]
расстояние между реальной и предполагаемой точками
 2.6709392194085


In [32]:
print('Со смещениями + delta')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0+deltas)
print('разности в координатах\n', abs(required_coordinates - Ur_0-deltas))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0+deltas))

Со смещениями + delta
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 449.3194871  3636.33563387 5201.8191359 ]
разности в координатах
 [2.9691629  0.42247587 1.5787251 ]
расстояние между реальной и предполагаемой точками
 3.3892162980614353


## III способ расчета

Введем опорные расстояния - целочисленные части расстояний от каждого спутника до потребителя (станции)

In [0]:
Ro_0 = Ro.astype(int).astype(np.float64)

Ищем опорные координаты потребителя

In [34]:
%time Ur_0 = fsolve(to_solve_Ur_0, np.array([1.,1.,1.]), args=(list(zip(Us,Ro_0))))
Ur_0              

CPU times: user 1.81 ms, sys: 0 ns, total: 1.81 ms
Wall time: 1.93 ms


array([ 459.61470208, 3628.14362089, 5209.3568971 ])

Ищем направлющие косинусы

In [35]:
dir_cosines = [dir_cos(*vals) for vals in zip(Us,[Ur_0]*3,Ro_0)]
dir_cosines

[array([0.09055693, 0.61521979, 0.78313732]),
 array([-0.58634968, -0.0220541 ,  0.80975778]),
 array([0.71992072, 0.69279444, 0.0418333 ])]

Ищем дельты

In [36]:
%time deltas = fsolve(to_solve_deltas,[.0,.0,.0],args=(dir_cosines,Ro_0,Ro))
deltas

CPU times: user 1.2 ms, sys: 214 µs, total: 1.41 ms
Wall time: 3.04 ms


array([ 9.05382819, -9.64316787,  7.26578006])

смотрим результаты

In [37]:
print('Без смещений')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0)
print('разности в координатах\n', abs(required_coordinates - Ur_0))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0))

Без смещений
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 459.61470208 3628.14362089 5209.3568971 ]
разности в координатах
 [7.32605208 7.76953711 5.9590361 ]
расстояние между реальной и предполагаемой точками
 12.228935242532412


In [38]:
print('Со смещениями - delta')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0-deltas)
print('разности в координатах\n', abs(required_coordinates - Ur_0+deltas))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0-deltas))

Со смещениями - delta
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 450.56087389 3637.78678876 5202.09111704]
разности в координатах
 [1.72777611 1.87363076 1.30674396]
расстояние между реальной и предполагаемой точками
 2.8641372644435443


In [39]:
print('Со смещениями + delta')
print('искомое\n', required_coordinates)
print('полученное\n', Ur_0+deltas)
print('разности в координатах\n', abs(required_coordinates - Ur_0-deltas))
print('расстояние между реальной и предполагаемой точками\n', distance(required_coordinates,Ur_0+deltas))

Со смещениями + delta
искомое
 [ 452.28865  3635.913158 5203.397861]
полученное
 [ 468.66853027 3618.50045303 5216.62267716]
разности в координатах
 [16.37988027 17.41270497 13.22481616]
расстояние между реальной и предполагаемой точками
 27.320295284129973


In [40]:
np.array([ 452.28865,  -3635.913158, 5203.397861]).astype(int).astype(np.float64)

array([  452., -3635.,  5203.])

## Результаты

Дельта отображается без косяков

In [41]:
print('Δ','δ')

Δ δ


Напишем функции, соответствующие 1,2,3-му методам.

На входе у них список выборок из таблиц (данных о спутниках в опр момент времени)

На выходе - найденные координаты потребителя и дельты

In [0]:
def ingot():
    return    {'Xr без коррекции':[],
               'Yr без коррекции':[],
               'Zr без коррекции':[],
               'ΔX':[],
               'ΔY':[],
               'ΔZ':[],}

In [0]:
def method_1(samples: list):
    res = ingot()
    for sample in samples:
        sat_data = [get_Us_and_P(sample.iloc[i]) for i in range(3)]
        Ur_0 = fsolve(to_solve_Ur_0, np.array([1.,1.,1.]), args=(sat_data))
        Ro = np.array([P_i for Us_i,P_i in sat_data])
        Ro_0 = np.floor(Ro)
        Us = [Us_i for Us_i,P in sat_data]
        dir_cosines = [dir_cos(*vals) for vals in zip(Us,[Ur_0]*3,Ro_0)]
        deltas = fsolve(to_solve_deltas,[.0,.0,.0],args=(dir_cosines,Ro_0,Ro))
        for i,ax in enumerate(('X','Y','Z')):
           res[ax+'r без коррекции'].append(Ur_0[i])
           res['Δ'+ax].append(deltas[i])
    return res

In [0]:
def method_2(samples: list):
    res = ingot()
    for sample in samples:
        sat_data = [get_Us_and_P(sample.iloc[i]) for i in range(3)]
        Ro = np.array([P_i for Us_i,P_i in sat_data])
        Us = [Us_i for Us_i,P in sat_data]
        Ur_0 = np.floor(fsolve(to_solve_Ur_0, np.array([1.,1.,1.]), args=(sat_data)))
        Ro_0 = np.array([distance(Ur_0,Us_i) for Us_i in Us])
        dir_cosines = [dir_cos(*vals) for vals in zip(Us,[Ur_0]*3,Ro_0)]
        deltas = fsolve(to_solve_deltas,[.0,.0,.0],args=(dir_cosines,Ro_0,Ro))
        for i,ax in enumerate(('X','Y','Z')):
           res[ax+'r без коррекции'].append(Ur_0[i])
           res['Δ'+ax].append(deltas[i])
    return res

In [0]:
def method_3(samples: list):
    res = ingot()
    for sample in samples:
        sat_data = [get_Us_and_P(sample.iloc[i]) for i in range(3)]
        Ro = np.array([P_i for Us_i,P_i in sat_data])
        Us = [Us_i for Us_i,P in sat_data]
        Ro_0 = np.floor(Ro)
        Ur_0 = fsolve(to_solve_Ur_0, np.array([1.,1.,1.]), args=(list(zip(Us,Ro_0))))
        dir_cosines = [dir_cos(*vals) for vals in zip(Us,[Ur_0]*3,Ro_0)]
        deltas = fsolve(to_solve_deltas,[.0,.0,.0],args=(dir_cosines,Ro_0,Ro))
        for i,ax in enumerate(('X','Y','Z')):
           res[ax+'r без коррекции'].append(Ur_0[i])
           res['Δ'+ax].append(deltas[i])
    return res

In [0]:
from random import sample as some_samples

Выберем список из 12 моментов времени, глянем превью

In [47]:
n_sample_time = some_samples(observed_more_than_two.index.to_list(), 12)
samples = [mod_data[mod_data['Time_mom'] == sample_time] for sample_time in n_sample_time]
samples[4:8]

[     Time_mom  N_Sat            P1         Sat_X         Sat_Y         Sat_Z
 989  05:59:00      2  19229.039986  -2387.906537  11335.717867  22593.069238
 990  05:59:00      1  22502.501863 -16551.387248   -453.214231  19363.615199
 991  05:59:00      3  20625.332488  14322.636232  16923.566654  12717.123746,
      Time_mom  N_Sat            P1         Sat_X         Sat_Y         Sat_Z
 854  05:36:30      2  19047.152234    775.188087  13961.171437  21205.824546
 855  05:36:30      1  21555.207230 -13483.185776   1797.208424  21544.902119
 856  05:36:30      3  21419.485318  15767.870829  18252.538300   8456.348608,
       Time_mom  N_Sat            P1         Sat_X         Sat_Y         Sat_Z
 1380  07:04:30      2  20922.612073 -13202.848088   5713.618322  20918.765632
 1381  07:04:30      3  19322.975899   5987.220552  12559.046839  21424.336160
 1382  07:04:30      4  21869.145032  19894.414228  12608.552221   9648.054238,
       Time_mom  N_Sat            P1         Sat_X       

составим удобоваримые таблички для результатов расчета тремя методами по 12 образцов каждый

In [0]:
results = [pd.DataFrame(eval('method_'+str(i+1)+'(samples)')) for i in range(3)]

for df in results:
    to_pop = list()
    for sign in ('+','-'):
        for i,ax in enumerate(('X','Y','Z')):
            tmp = eval("df[ax+'r без коррекции']"+sign+
                       "df['Δ'+ax] - required_coordinates[i]" )
            df['{a}r {s} Δ{a} - {a} реал.'.format(a=ax,s=sign)] = tmp
            hs = str(i)+sign
            to_pop.append(hs)
            df[hs] = tmp**2
        df['Расстояние от реал до (Ur{s}Δ)'.format(s=sign)] = np.sqrt(
            df['0'+sign]+df['1'+sign]+df['2'+sign]
        )
    for hs in to_pop:
        df.pop(hs)
    to_pop.clear()

In [49]:
results[0]

,Xr без коррекции,Yr без коррекции,Zr без коррекции,ΔX,ΔY,ΔZ,Xr + ΔX - X реал.,Yr + ΔY - Y реал.,Zr + ΔZ - Z реал.,Расстояние от реал до (Ur+Δ),Xr - ΔX - X реал.,Yr - ΔY - Y реал.,Zr - ΔZ - Z реал.,Расстояние от реал до (Ur-Δ)
0,187.033453,4390.551656,4899.142178,21.719833,-62.496578,23.746687,-243.535365,692.141920,-280.508995,785.528617,-286.975030,817.135076,-328.002370,926.093923
1,452.217320,3637.885684,5202.530910,1.121106,-18.963251,8.973653,1.049776,-16.990725,8.106702,18.854850,-1.192437,20.935776,-9.840605,23.163897
2,452.884814,3634.830897,5203.876359,-6.326843,12.263342,-5.685782,-5.730679,11.181081,-5.207284,13.600480,6.923006,-13.345603,6.164280,16.249046
3,453.103848,3634.015538,5204.193854,-2.046376,5.604268,-1.835282,-1.231178,3.706648,-1.039290,4.041678,2.861574,-7.501888,2.631275,8.449292
4,450.889587,3638.069510,5202.168596,17.201832,-25.423317,14.105143,15.802769,-23.266965,12.875879,30.933274,-18.600895,27.579669,-15.334408,36.630254
5,450.819970,3637.716767,5202.228786,13.604227,-15.794098,10.093762,12.135547,-13.990489,8.924687,20.558583,-15.072907,17.597707,-11.262837,25.762828
6,210.281750,4335.417911,4920.509242,-7.395363,22.155377,-7.308530,-249.402263,721.660129,-290.197149,816.828756,-234.611537,677.349376,-275.580089,767.977302
7,163.284556,4445.452919,4877.961314,6.771102,-20.371702,8.249860,-282.232992,789.168059,-317.186687,896.130059,-295.775196,829.911462,-333.686407,942.116033
8,450.621827,3637.671201,5202.153284,36.496748,-37.488674,26.301766,34.829925,-35.730631,25.057189,55.836049,-38.163571,39.246716,-27.546342,61.282655
9,450.892094,3638.076712,5202.167899,-2.589598,4.223653,-1.377652,-3.986154,6.387207,-2.607614,7.967778,1.193042,-2.060100,0.147690,2.385199


In [50]:
results[1]

,Xr без коррекции,Yr без коррекции,Zr без коррекции,ΔX,ΔY,ΔZ,Xr + ΔX - X реал.,Yr + ΔY - Y реал.,Zr + ΔZ - Z реал.,Расстояние от реал до (Ur+Δ),Xr - ΔX - X реал.,Yr - ΔY - Y реал.,Zr - ΔZ - Z реал.,Расстояние от реал до (Ur-Δ)
0,187.0,4390.0,4899.0,-0.033301,-0.552095,-0.142007,-265.321951,753.534747,-304.539868,854.958995,-265.255349,754.638937,-304.255854,855.810697
1,452.0,3637.0,5202.0,-0.217298,-0.886119,-0.530710,-0.505948,0.200723,-1.928571,2.003911,-0.071352,1.972961,-0.867151,2.156297
2,452.0,3634.0,5203.0,-0.885536,-0.829567,-0.876970,-1.174186,-2.742725,-1.274831,3.244449,0.596886,-1.083591,0.479109,1.326645
3,453.0,3634.0,5204.0,-0.103869,-0.015487,-0.193876,0.607481,-1.928645,0.408263,2.062858,0.815219,-1.897671,0.796015,2.213453
4,450.0,3638.0,5202.0,-0.889689,-0.069357,-0.168657,-3.178339,2.017485,-1.566518,4.077507,-1.398961,2.156199,-1.229204,2.849075
5,450.0,3637.0,5202.0,-0.819977,-0.716765,-0.228759,-3.108627,0.370077,-1.626620,3.527947,-1.468673,1.803607,-1.169102,2.603228
6,210.0,4335.0,4920.0,-0.281429,-0.418869,-0.508876,-242.570079,698.667973,-283.906737,792.199605,-242.007221,699.505711,-282.888985,792.402620
7,163.0,4445.0,4877.0,-0.283830,-0.455012,-0.960531,-289.572480,808.631830,-327.358392,919.185060,-289.004820,809.541854,-325.437330,919.125266
8,450.0,3637.0,5202.0,-0.621652,-0.671384,-0.153140,-2.910302,0.415458,-1.551001,3.323864,-1.666998,1.758226,-1.244721,2.723889
9,450.0,3638.0,5202.0,-0.892201,-0.076550,-0.167964,-3.180851,2.010292,-1.565825,4.075647,-1.396449,2.163392,-1.229897,2.853591


In [51]:
results[2]

,Xr без коррекции,Yr без коррекции,Zr без коррекции,ΔX,ΔY,ΔZ,Xr + ΔX - X реал.,Yr + ΔY - Y реал.,Zr + ΔZ - Z реал.,Расстояние от реал до (Ur+Δ),Xr - ΔX - X реал.,Yr - ΔY - Y реал.,Zr - ΔZ - Z реал.,Расстояние от реал до (Ur-Δ)
0,210.905955,4321.932319,4925.357779,26.495830,-76.080742,29.224377,-214.886865,609.938419,-248.815705,692.900061,-267.878524,762.099903,-307.264459,864.272303
1,453.351541,3618.723116,5211.601925,1.147580,-19.365474,9.170132,2.210470,-36.555516,17.374196,40.534610,-0.084689,2.175433,-0.966068,2.381799
2,446.519832,3647.172856,5198.159686,-6.403443,12.421260,-5.747809,-12.172261,23.680957,-10.985984,28.803533,0.634624,-1.161562,0.509634,1.418345
3,451.047561,3639.644925,5202.348653,-2.066356,5.654881,-1.855292,-3.307444,9.386648,-2.904500,10.367472,0.825267,-1.923114,0.806084,2.242588
4,468.551844,3611.982478,5216.677261,18.147567,-26.786605,14.934016,34.410761,-50.717285,28.213416,67.471033,-1.884373,2.855925,-1.654616,3.800648
5,464.683763,3621.624117,5212.527285,14.133215,-16.402537,10.511027,26.528328,-30.691578,19.640451,45.071859,-1.738103,2.113495,-1.381604,3.065402
6,203.102738,4356.948144,4913.453624,-6.974099,20.938065,-6.816155,-256.160011,741.973051,-296.760392,839.171430,-242.211813,700.096921,-283.128082,793.072362
7,170.245815,4424.548638,4886.425292,7.161226,-21.464245,8.689125,-274.881609,767.171235,-308.283444,871.292307,-289.204060,810.099725,-325.661694,919.758714
8,489.209281,3598.107594,5230.004006,40.928008,-41.886422,29.584822,77.848639,-79.691986,56.190967,124.774388,-4.007377,4.080859,-2.978677,6.448643
9,448.312245,3642.286256,5200.798845,-2.569664,4.194774,-1.360137,-6.546069,10.567872,-3.959153,13.046296,-1.406741,2.178324,-1.238879,2.873819
